## [Lab 4 (Part I)] BLENDS Node

In Lab #4 you will work in BLENDS, a simple cryptocurrency framework we have built for CS443. The BLENDS framework provides an overall architecture with well-defined APIs so that students can focus on core concepts of Bitcoin-like cryptocurrency and do not worry about architecting the system from scratch.

The framework makes the following assumptions:
* All nodes communicate not directly with each other, but through a broadcast server provided in the framework.
* Unless Bitcoin, here a single (priv, pub) key pair is associated with a node.  That is, we equate the concept of a pub key with a node.
* We use RSA, instead of ECDSA for public key crypto.
* We use SHA-512, instead of SHA-256. You will be using RSASSA-PKCS1_v1_5 of PyCryptodome [Modified on 2018/12/12]
* We do not use Merkle tree, but just a simple linked list with hash pointers.
* For ease of programming and balance-checking, we use an RDBMS called SQLite to store transactions and blocks.

The broadcast server of the framework does the following:
* It works as a communication center by accepting transactions and blocks from a node and broadcasting them to other nodes.
* It drops transactions and blocks if they are formatted correctly.
* [Under Construction] It runs a leaderboard.

A node in the framework is architected as follows:
* It is made up of two processes: main and CLI.
* The main process forks a miner thread in order to mine continuously.
* The CLI process is the command-line interface for typing in commands to be delivered to the node.  It is given in "cli.py".
* The CLI process communicates with the main process over HTTP.  "manager.py" implements the interface between the CLI and main processes.  This is also given.
* The main process is broken into 7 files: blockchain.py, db.py, node.py, crypto.py, miner.py, util.py, and manager.py. util.py and manager.py are given.  For the rest, students will have to fill in the function body given the specification.
 
Now Part I of Lab 4.  First, let's start with digital signature.


In [ ]:
import json
from blends.node.crypto import load_secret_key, sign, verify, get_pk

MESSAGE = "This is a test message"

## Digital Signature: crypto.py

You are asked to implement the following functions:
* Create and save to a file an RSA secret key.
* Load an RSA secret key from a file.
* Get a public key from an RSA secret key.
* Sign a message with an RSA secret key.

We use RSA-2048 for digital signature. The parameter `publicExponent` is fixed at 0x10001 for all keys in this project.

### Problem 1 : create_secret_key(fname)

Implement `create_secret_key` function in [crypto.py](/edit/node/crypto.py)

* It creates an RSA key and store it in a file, "fname". We keep it in a file so that when you stop in the middle and return, your node can continue with the same key.  You can look at the file format in "key0.json".


### Problem 2 : load_secret_key(fname)

Implement `load_secret_key` function in [crypto.py](/edit/node/crypto.py)

* it returns secret key.
* it will be used in sign function below.

### Problem 3 : get_pk(sk)

Implement `get_pk` function in [crypto.py](/edit/node/crypto.py)

* It returns public key from secret key.

### Problem 4 : sign(sk, message)

Implement `sign` function in [crypto.py](/edit/node/crypto.py)

* Use SHA-512 to convert the message to a hash digest. Use UTF-8 encoding for the message.
* Sign message, i.e. encrypt the digest with your secret key, `sk`.
* `sk`  is return value of `load_secret_key` above.

In [ ]:
# Test  for Problems 2 to 4; Problem 1 is not tested here.

for i in range(3):
    key_fname = "key{}.json".format(i)
    sk = load_secret_key(key_fname)
    sig_computed = sign(sk, MESSAGE)
    sig_fname = "sig{}.json".format(i)
    with open(sig_fname, "r") as sig_f:
        data = json.loads(sig_f.readline())
        sig = data["sig"]
        if sig_computed == sig:
            print("PASS")
        else:
            print("FAIL")

### Problem 5 : verify(pk, msg, sig)

Next, Implement `verify` function in [crypto.py](/edit/node/crypto.py)

* it takes a public key `pk`, a message `msg`, and a signature `sig`.


In [ ]:
# Test  for Problem 5
WRONG_SIG = "wrong signature"

for i in range(3):
    key_fname = "key{}.json".format(i)
    sk = load_secret_key(key_fname)
    pk = get_pk(sk)
    sig_fname = "sig{}.json".format(i)
    with open(sig_fname, "r") as sig_f:
        data = json.loads(sig_f.readline())
        sig = data["sig"]
        if verify(pk, MESSAGE, sig):
            print("PASS")
        else:
            print("FAIL")


### Problem 6 : get_hash(msg)

To send a block to a peer, you need to format the hash digest into a hexadecimal number string.

* Implement `get_hash` function in [crypto.py](/edit/node/crypto.py).
